In [ ]:
import requests
import time
from bs4 import BeautifulSoup as Soup
import pandas as pd
import hashlib
import lxml
from datetime import datetime, date
import re
import ast
!pip install demjson
import demjson

     |████████████████████████████████| 131 kB 7.4 MB/s 
  Created wheel for demjson: filename=demjson-2.2.4-py3-none-any.whl size=73564 sha256=3026f02c7f877bcf5f77a1d6450415183f400e3dafd9df0985c66e81cd84e7b5
  Stored in directory: /root/.cache/pip/wheels/41/94/3d/466801f4a8db8e6fce765d7a0115dfebcc55ddf6b00cd98f59
Successfully built demjson


In [ ]:
def parse_sitemap(url, headers):
    # First of all we make a request to the specified url in the function parameters.
    resp = requests.get(url)
    # if we didn't get a valid response, return false 
    status = resp.status_code 
    if (status != 200):
        return False
    # Then we parse the content of the response using BeautifulSoup4.
    soup = Soup(resp.content, 'xml')

    #Then we look for either a urlset or a sitemapindex
    urls = soup.findAll('url')
    sitemaps = soup.findAll('sitemap')
    #create pandas dataframe
    new_list = ['Source'] + headers
    panda_out_total = pd.DataFrame([], columns=new_list)


    if not urls and not sitemaps:
        return False

    # Recursive call to the the function if sitemap contains sitemaps
    if sitemaps:
        for url in sitemaps:
            sitemap_url = url.find('loc').string
            panda_recursive = parse_sitemap(sitemap_url, headers)
            panda_out_total = pd.concat([panda_out_total, panda_recursive], ignore_index=True)

    # storage for later...
    out = []

    # Creates a hash of the parent sitemap for faster indexing
    hash_sitemap = hashlib.md5(str(url).encode('utf-8')).hexdigest()

    # Extract the keys we want
    for u in urls:
        values = [hash_sitemap]
        for head in headers:
            loc = None
            loc = u.find(head)
            if not loc:
                loc = 'None'
            else:
                loc = loc.string
            values.append(loc)
        out.append(values)
    
    # Creates a dataframe
    panda_out = pd.DataFrame(out, columns=new_list)

    # If recursive then merge recursive dataframe
    if not panda_out_total.empty:
        panda_out = pd.concat([panda_out, panda_out_total], ignore_index=True)

    #returns the dataframe
    return panda_out

In [ ]:
#Do not run this code section if the CSV file is on the drive
#result = parse_sitemap("https://www.shein.com/sitemap-index.xml", ["loc"])
#result.to_csv('shein_urls.csv')
urls = pd.read_csv('shein_urls.csv', index_col=0)

In [5]:
import csv

csv_file=open('OpsCons_Shein_table.csv','w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['id_style', 'display_name', 'materials', 'color', 'size', 'price', 'currency', 'product_url', 'image_links', 'brand_name', 'retailer', 'description', 'scrapped_date', 'high_level', 'low_level', 'gender', 'secondhand', 'shipping_from', 'style', 'availability'])
i=0
def extract_product_info(url):
        response = requests.get(url)

        results_page = Soup(response.content, 'html')
        scripts = results_page.find_all('script')
        for index, script in enumerate(scripts):
            text = script.text
            pattern = r'productIntroData: ([\s\S]*),\n[\s\S*]'
            product = re.findall(pattern, text)
            if product != []:
              test = product[0].replace('\n', '')
              d = demjson.decode(test)
              display_name = float('NaN')
              materials = float('NaN')
              color = float('NaN')
              size = float('NaN')
              availability = float('NaN')
              price = d['detail']['salePrice']['amount']
              currency = 'USD'
              product_url = url
              image_links = float('NaN')
              brand_name = 'Shein'
              retailer = 'Shein'
              description = float('NaN')
              scrapped_date = date.today()
              high_level = float('NaN')
              low_level = float('NaN')
              gender = float('NaN')
              secondhand = 'No'
              shipping_from = float('NaN')
              style = float('NaN')
              id_style=float('NaN')
              try:
                display_name = d['detail']['goods_name']
                materials = d['detail']['productDetails'][15]['attr_value']
                color = d['detail']['productDetails'][0]['attr_value']
                size = [d['sizeInfoDes']['sizeInfo'][i]['size'] for i in range(len(d['sizeInfoDes']['sizeInfo']))]
                image_links = [d['detail']['original_img'][2:]]
                description = d['metaInfo']['meta_description']
                high_level = d['currentCat']['cat_name']
                low_level = d['currentCat']['cat_name']
                gender = d['parentCats']['cat_name'].split(' ')[0]
                style = d['detail']['productDetails'][1]['attr_value']
              except Exception as e:
                pass

              csv_writer.writerow([id_style, display_name, materials, color, size, price, currency, product_url, image_links, brand_name, retailer, description, scrapped_date, high_level, low_level, gender, secondhand, shipping_from, style, availability])

url_list = urls['loc']
for url in url_list:
  extract_product_info(url)
csv_file.close()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
2869
2870
2871
2872
2873
2874
2875
2876
2877
2878
2879
2880
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
29

KeyboardInterrupt: ignored

## Data Cleaning


In [ ]:
import pandas as pd

df = pd.read_csv('OpsCons_Shein_table.csv')
df.drop_duplicates(keep = 'first', inplace=True)
df['color'] = df['color'].str.replace('[','{').str.replace(']','}')

In [ ]:
GranCat = pd.read_csv("LowLevelCatagoryRegex.csv", header=None, skiprows=[0])

def process_cat(df):
    # get all the values in category regex
    subCats = GranCat.iloc[:, 1:]
    Cats = []
    for index, row in subCats.iterrows():
        for i in GranCat.columns[1:]:
            if pd.isnull(row[i]): continue
            else: Cats.append(row[i])
               
    # assign potential category for each clothing row based on title, else other
    clothesCat = []
    for index, row in df.iterrows():
        rowCat = float("NaN")
        for cat in Cats:
            if re.search(cat, row['display_name'].lower()):
                rowCat = cat
                print(rowCat)
                break
        if pd.isnull(rowCat):
            clothesCat.append('other')
        else:
            clothesCat.append(rowCat)

    return clothesCat

# set parent category to each product
def get_cat(x):
    if x == 'other': return x
    else: return [GranCat[0][i] for i in GranCat.index if x in GranCat.iloc[i].to_list()][0]

# get low_level column given a df
def get_lowlevel(df):
    df['low_level'] = process_cat(df)
    df['low_level'] = df['low_level'].apply(lambda x: get_cat(x))
    return df

df = get_lowlevel(df)

jumpsuits?
tops?
bracelets?
tees?
rings?
sweaters?
tank
pullover
sweatshirts?
swimsuits?
sets?$
necklaces?
tees?
sweaters?
coats?
dresse?s?$
tees?
dresse?s?$
rings?
dresse?s?$
purse
necklaces?
hoodies?
hoodies?
shirts?
dresse?s?$
dresse?s?$
jumpsuits?
blouse
sk[io]rts?
jumpsuits?
sweaters?
bands?
sweatshirts?
sets?$
tees?
watch
blouse
dresse?s?$
pullover
shirts?
scarf?v?e?s?
tees?
tops?


In [ ]:
#High level

import re
from math import *
GranCat = pd.read_csv("HighLevelCatagory-LowLevelCatagory - HighLevelLowLevel.csv" , header=None, skiprows=[0])


def process_cat(df1,df2):
    subCats = GranCat.iloc[:, 0:2]
    for i in range(len(df2)):
      df1[i]=df2[i]
      for index2, row2 in subCats.iterrows():
        if df2[i]==row2[1]:
          df1[i]=row2[0]
    return df1
    
               

# get high_level column given a df
def get_highlevel(df):
    df['high_level']=process_cat(df['high_level'],df['low_level'])
    return df

df = get_highlevel(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
##### get mapped_id
GranRegs = pd.read_csv('MaterialProxy.csv', header=None, skiprows=[0])
GranRegs = GranRegs.drop(columns=[1])
GranRegs.columns = list(range(len(GranRegs.columns)))
#extract materials

#extract materials

def extractMaterials(df):
    fabric_text = df['materials']
    fabric_text = str(fabric_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_fabric = fabric_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '').split()
    pattern = "\d{1,3}[$%]"
    
    description_text = str(df['description']) + str(df['display_name']) + str(df['materials'])
    description_text = str(description_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_description = description_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '')
    pattern = "\d{1,3}[$%]"
    
    mat_list = []
    extra_materials = []
    
    
    ## getting the materials regex matching
    subRegs = GranRegs.iloc[:, 1:]
    Regs = []
    materialsReg = []
    for index, row in subRegs.iterrows():
        for i in range(1,len(GranRegs.columns)):
            if pd.isnull(row[i]): continue
            else: Regs.append(row[i])
    total_percentage = [] 
    is_no_percentage = True 
    for i in range(len(splitted_fabric)):
        if re.fullmatch(pattern,splitted_fabric[i]):
            total_percentage.append(float(splitted_fabric[i].replace("%", '')))
            if sum(total_percentage) <= 100:
            
                if splitted_fabric[i+1].lower() == 'organic':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                elif splitted_fabric[i+1].lower() == 'recycled':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                
                elif splitted_fabric[i+1].lower() == 'tencel™':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                else:
                    if splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() not in mat_list:
                        mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower())
                        is_no_percentage = False
                        
    if sum(total_percentage) < 100 or is_no_percentage == True:
        for reg in Regs:
            #print(splitted_description.lower())
            if re.search(reg, splitted_description.lower()):
                match = re.search(reg, splitted_description.lower())
                matched_material = splitted_description[match.start():match.end()].lower()
                
                if matched_material not in extra_materials:
                    extra_materials.append(matched_material)
    
    #print(extra_materials)
    if len(extra_materials)>1:
        if "cotton" in extra_materials and "organic cotton" in extra_materials:
            mat_list.append("100% organic cotton")
        if "leather" in extra_materials:
            mat_list.append("100% leather")
    elif len(extra_materials)!=0:
        mat_list.append("100% " + extra_materials[0])

            
    
    return mat_list


In [ ]:
df['materials'] = df.apply(lambda x: extractMaterials(x), axis=1)

In [ ]:
df.to_csv('OpsCons_Shein_table.csv', encoding='utf-8-sig')

In [ ]:
df

,id_style,display_name,materials,color,size,price,currency,product_url,image_links,brand_name,retailer,description,scrapped_date,high_level,low_level,gender,secondhand,shipping_from,style
0,NaN,SHEIN Off Shoulder Wrap Cross Unitard Jumpsuit...,"[9% elastane, 100% elastane]",Elegant,"['XS', 'S', 'M', 'L', 'XL']",17.00,USD,https://www.shein.com/Off-Shoulder-Wrap-Cross-...,['img.ltwebstatic.com/images3_pi/2021/10/09/16...,Shein,Shein,To find out about the SHEIN Off Shoulder Wrap ...,2021-11-18,rompers,rompers,Women,No,NaN,Black
1,NaN,Gigot Sleeve Wrap Knot Side Top,"[5% elastane, 100% elastane]",Black,"['XS', 'S', 'M', 'L']",13.00,USD,https://www.shein.com/Gigot-Sleeve-Wrap-Knot-S...,['img.ltwebstatic.com/images3_pi/2021/10/23/16...,Shein,Shein,To find out about the Gigot Sleeve Wrap Knot S...,2021-11-18,shirts,shirts,Women,No,NaN,Sexy
2,NaN,2pcs Faux Pearl Decor Bracelet,[],NaN,NaN,3.00,USD,https://www.shein.com/2pcs-Faux-Pearl-Decor-Br...,NaN,Shein,Shein,NaN,2021-11-18,Jewelry,Jewelry,NaN,No,NaN,NaN
3,NaN,Plus Contrast Trim Petal Sleeve Tee,[],NaN,NaN,13.00,USD,https://www.shein.com/Plus-Contrast-Trim-Petal...,NaN,Shein,Shein,NaN,2021-11-18,shirts,shirts,NaN,No,NaN,NaN
4,NaN,SHEIN Teddy Waterfall Collar Drawstring Cuff W...,[100% polyester],Beige,"['XS', 'S', 'M', 'L']",44.00,USD,https://www.shein.com/Teddy-Waterfall-Collar-D...,['img.ltwebstatic.com/images3_pi/2021/10/08/16...,Shein,Shein,To find out about the SHEIN Teddy Waterfall Co...,2021-11-18,Jewelry,Jewelry,Women,No,NaN,Casual
5,NaN,Turtleneck Drop Shoulder Fuzzy Sweater,[],Casual,"['S', 'M', 'L']",24.00,USD,https://www.shein.com/Turtleneck-Drop-Shoulder...,['img.ltwebstatic.com/images3_pi/2021/09/27/16...,Shein,Shein,To find out about the Turtleneck Drop Shoulder...,2021-11-18,sweaters,sweaters,Women,No,NaN,Apricot
6,NaN,Plus Single Breasted Tank Jumpsuit,[],Black,"['0XL', '1XL', '2XL', '3XL', '4XL']",17.49,USD,https://www.shein.com/Plus-Single-Breasted-Tan...,['img.ltwebstatic.com/images3_pi/2021/07/23/16...,Shein,Shein,To find out about the Plus Single Breasted Tan...,2021-11-18,shirts,shirts,Women,No,NaN,Casual
7,NaN,Toddler Girls Fox Print Pullover,[],Multicolor,"['80', '90', '100', '110']",15.00,USD,https://www.shein.com/Toddler-Girls-Fox-Print-...,['img.ltwebstatic.com/images3_pi/2021/10/19/16...,Shein,Shein,To find out about the Toddler Girls Fox Print ...,2021-11-18,sweaters,sweaters,Kids,No,NaN,Casual
8,NaN,Plus Pearls Beaded Lantern Sleeve Sweatshirt D...,[],Black,"['0XL', '1XL', '2XL', '3XL', '4XL']",22.00,USD,https://www.shein.com/Plus-Pearls-Beaded-Lante...,['img.ltwebstatic.com/images3_pi/2021/09/01/16...,Shein,Shein,To find out about the Plus Pearls Beaded Lante...,2021-11-18,sweatshirts,sweatshirts,Women,No,NaN,Casual
9,NaN,Striped Push Up Bikini Swimsuit,[],NaN,NaN,15.00,USD,https://www.shein.com/Striped-Push-Up-Bikini-S...,NaN,Shein,Shein,NaN,2021-11-18,swimwear,swimwear,NaN,No,NaN,NaN
